# Будем парсить ozon на предмет пластика для 3д принтеров

In [764]:
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import pandas as pd
import time
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
import random
import re


функция генерации ссылки на лист с категорией пластиков

In [765]:

def generate_paged_url_for_goods(page_number=1, base_url='https://www.ozon.ru/category/plastik-dlya-3d-printera-15794/?opened=typeofplastic&sorting=rating&type=52987&typeofplastic=285449%2C100857334%2C336372%2C336371%2C336375%2C336369%2C285451%2C100857335%2C285452%2C336370'):
    # Разбираем базовый URL
    parsed_url = urlparse(base_url)

    # Извлекаем параметры запроса
    query_parameters = parse_qs(parsed_url.query)
 
    # Устанавливаем номер страницы
    query_parameters['page'] = [str(page_number)]

    # Преобразуем обновленные параметры запроса обратно в строку
    updated_query_string = urlencode(query_parameters, doseq=True)

    # Обновляем URL с новыми параметрами запроса
    updated_url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path, parsed_url.params, updated_query_string, parsed_url.fragment))

    return updated_url

подготовим функцию для обрезания ссылки

In [766]:
def remove_query_parameters(url):
    # Parse the URL into its components
    parsed_url = urlparse(url)
    
    # Reconstruct the URL without query parameters
    clean_url = urlunparse(parsed_url._replace(query=''))
    
    return clean_url

так как на озоне много непонятных имен классов - будем их выносить сюда, чтоб если вдруг они сменились - оперативно и быстро их менять

In [767]:
# контейнер с товаром
paged_good_container="u9i"
# параметры товара
characteristics_class="jt0"
characteristics_parameter_name_class="sj9"
characteristics_parameter_value_class="s9j"
rating_class="e1317-b.e1317-a2"
review_count_class="pt7"
name_class="ll2"
price_class="zk7"

создадим функции рандомов для паузы и скролинга

In [768]:
def random_sleep_time():
    return round(random.uniform(0.1, 0.3), 2)
def random_scroll():
    return random.randint(500, 1500)
def scroll(browser,i=10):
    scroll_pos=0
    for i in range(0, i):    
        time.sleep(random_sleep_time())
        rs=random_scroll()
        script = f"window.scrollBy({scroll_pos}, {rs});"
        browser.execute_script(script)
        scroll_pos+=rs

Подключим драйвер chrome и открываем страницу(и получаем ссылки)

In [769]:
options = webdriver.ChromeOptions() 
options.add_argument("--disable-popup-blocking")
options.headless = False
browser = uc.Chrome(options=options)
links=[]
#всего есть 150 страниц, но думаю для сета хватит и 50 с головой
for i in range(1,100):
    browser.get(generate_paged_url_for_goods(i))
    scroll(browser)
    goods=browser.find_elements(By.CLASS_NAME,paged_good_container)
    len(goods)
    for good in goods:
        link=good.find_element(By.TAG_NAME,'a')
        link.get_attribute('href')
        full_link=link.get_attribute('href')
        link=remove_query_parameters(full_link)
        links.append(link)

подготовим запрос, который будет возвращать нам описание и запрос, который ведет к отзывам


In [770]:
def get_data(link):
    # грузить всю страницу и немного поскролить оказалось быстрей чем грузить блоки по отдельности, что в принципе логично
    browser.get(link)
    scroll(browser,3)
    table={}
    table.update(get_name(browser))
    table.update(get_description(browser))
    table.update(get_price(browser))
    table.update(get_reviews_count(browser))
    table.update(get_rating(browser))
    table.update(get_characteristics(browser))
    return table
def get_rating(browser: uc):
    rating=browser.find_element(By.CLASS_NAME,rating_class)
    pattern = r'\d+(\.\d+)?'
    matches = re.search(pattern, rating.get_attribute('style'))
    return{"rating":float(matches.group())}
def get_name(browser: uc):
    name=browser.find_element(By.CLASS_NAME,name_class)
    return{"name":name.text}
def get_reviews_count(browser: uc):
    reviews=browser.find_element(By.CLASS_NAME,review_count_class)
    return {"reviews_count":re.sub(r'\D', '', reviews.text)}
def get_price(browser: uc):
    price=browser.find_element(By.CLASS_NAME,price_class)
    return {"price":re.sub(r'\D', '', price.text)}
def get_description(browser: uc):
    description=browser.find_element(By.ID,"section-description")
    return {"description":description.text}
def get_characteristics(browser: uc):
    characteristics=browser.find_element(By.ID,"section-characteristics")
    parameters=characteristics.find_elements(By.CLASS_NAME,characteristics_class)
    table={}
    for parameter in parameters:
        name=parameter.find_element(By.CLASS_NAME,characteristics_parameter_name_class)
        val=parameter.find_element(By.CLASS_NAME,characteristics_parameter_value_class)
        table[name.text]=val.text
    return table
    # print(description.get_attribute('innerHTML'))

In [771]:
result_table=pd.DataFrame()
i=0
for link in links:
    i+=1
    print(f"обрабатываю {i} ссылку из {len(links)}, осталось {len(links)-i}")
    time.sleep(random_sleep_time())
    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[1])
    try:
        table=get_data(link)
        df=pd.DataFrame([table])
        result_table=pd.concat([result_table,df],ignore_index=True)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
    except:
        print(f"{i}can't load link {link}")
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        time.sleep(60)
browser.close()



обрабатываю 1 ссылку из 3558, осталось 3557
обрабатываю 2 ссылку из 3558, осталось 3556
обрабатываю 3 ссылку из 3558, осталось 3555
обрабатываю 4 ссылку из 3558, осталось 3554
обрабатываю 5 ссылку из 3558, осталось 3553
обрабатываю 6 ссылку из 3558, осталось 3552
обрабатываю 7 ссылку из 3558, осталось 3551
обрабатываю 8 ссылку из 3558, осталось 3550
обрабатываю 9 ссылку из 3558, осталось 3549
обрабатываю 10 ссылку из 3558, осталось 3548
обрабатываю 11 ссылку из 3558, осталось 3547
обрабатываю 12 ссылку из 3558, осталось 3546
обрабатываю 13 ссылку из 3558, осталось 3545
обрабатываю 14 ссылку из 3558, осталось 3544
обрабатываю 15 ссылку из 3558, осталось 3543
обрабатываю 16 ссылку из 3558, осталось 3542
обрабатываю 17 ссылку из 3558, осталось 3541
обрабатываю 18 ссылку из 3558, осталось 3540
обрабатываю 19 ссылку из 3558, осталось 3539
обрабатываю 20 ссылку из 3558, осталось 3538
обрабатываю 21 ссылку из 3558, осталось 3537
обрабатываю 22 ссылку из 3558, осталось 3536
обрабатываю 23 ссыл

попробуем найти объекты которые нас интересуют

In [772]:
result_table.to_csv("before_eda.csv")